In [1]:
import pandas as pd
import anndata

## Parameters

In [2]:
use_clusters = [
    'CA1_Ak5', 'CA1_Chrm3', 'CA1_Kif26a', 'CA1_Ptprg',
    'CA3_Cadm2', 'CA3_Efnb2', 'CA3-St18_Epha5', 'CA3-St18_Nuak1',
    'CA3-St18_Tead1', 'DG_dg-all', 'DG-po_Bcl11a', 'DG-po_Calb2',
    'DG-po_Kctd8', 'Gfra1_Gfra1', 'IG-CA2_Chrm3', 'IG-CA2_Peak1', 'IG-CA2_Xpr1'
]
hypo_dmr_hits_path = '/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalHits.HypoDMR.h5ad'
# 'CA1_Lingo2' not include, because it co-cluster with RHP, probably an incomplete cluster

In [3]:
# Parameters
use_clusters = ["CA1_Ak5", "CA1_Chrm3", "CA1_Kif26a", "CA1_Lingo2", "CA1_Ptprg", "CA3_Cadm2", "CA3_Efnb2", "CA3-St18_Epha5", "CA3-St18_Nuak1", "CA3-St18_Tead1", "DG-po_Bcl11a", "DG-po_Calb2", "DG-po_Kctd8", "Gfra1_Gfra1", "IG-CA2_Chrm3", "IG-CA2_Peak1", "IG-CA2_Xpr1"]
hypo_dmr_hits_path = "/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalHits.HypoDMR.h5ad"


## Hypo Hits

In [4]:
hypo_hits_adata = anndata.read_h5ad(hypo_dmr_hits_path)
related_dmr = (hypo_hits_adata[:, use_clusters].X.sum(axis=1) > 0).A1
use_dmr = hypo_hits_adata.obs_names[related_dmr]

In [5]:
use_hits = hypo_hits_adata[use_dmr][:, use_clusters]
use_hits.write_h5ad('RelatedClusterAndDMR.hypo_hits.h5ad')

In [6]:
use_hits

AnnData object with n_obs × n_vars = 1728707 × 17 
    obs: '#chr', 'start', 'end', 'number_of_dms'

## DMR Rate

In [7]:
with pd.HDFStore('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRInfo.h5') as hdf:
    dmr_rate = hdf['Rate']
dmr_rate.index = hypo_hits_adata.obs_names
use_rate = dmr_rate.loc[use_dmr, use_clusters]

In [8]:
use_rate.to_msgpack('RelatedClusterAndDMR.mcg_rate.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  """Entry point for launching an IPython kernel.


In [9]:
with open('related_dmrs.txt', 'w') as f:
    for dmr in use_dmr:
        f.write(dmr + '\n')